Importing libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import string

ModuleNotFoundError: No module named 'pandas'

Loading the Dataset from the disk

In [ ]:
raw_train_ds = tf.keras.utils.text_dataset_from_directory('train',batch_size=32,validation_split=0.2,
                                                         subset = 'training',seed = 42)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [ ]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory('train',batch_size=32,validation_split=0.2,
                                                         subset = 'validation',seed = 42)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [ ]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory('test',batch_size=32)

Found 8000 files belonging to 4 classes.


Exploring the data

In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print("Question",text_batch.numpy()[i])
        print("Label",text_batch.numpy()[i])

Question b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default const

In [ ]:
print("Label 0 is for ",raw_train_ds.class_names[0])
print("Label 1 is for ",raw_train_ds.class_names[1])
print("Label 2 is for ",raw_train_ds.class_names[2])
print("Label 3 is for ",raw_train_ds.class_names[3])

Label 0 is for  csharp
Label 1 is for  java
Label 2 is for  javascript
Label 3 is for  python


Preprocessing the Data

In [ ]:
def standardizer(text):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text,'<br />',' ')
    text = tf.strings.regex_replace(text,'[%s]' % re.escape(string.punctuation),'')
    return text

In [ ]:
# Creating a Text Vectorizer

max_features = 10000
sequence_length = 250

vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_features,standardize=standardizer,
                                               output_mode='int',output_sequence_length=sequence_length)

In [ ]:
train_text = raw_train_ds.map(lambda x,y:x)
vectorizer.adapt(train_text)

In [ ]:
def vectorize_text(text,label):
    text = tf.expand_dims(text,-1)
    return vectorizer(text),label

In [ ]:
train_ds = raw_train_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

Creating Models

In [ ]:
embedding_dim = 16

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features+1,embedding_dim),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4)
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          160016    
                                                                 
 dropout_2 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 68        
                                                                 
Total params: 160,084
Trainable params: 160,084
Non-trainable params: 0
________________________________________________

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),optimizer='adam',metrics=['accuracy'])

Training the Model

In [ ]:
model.fit(train_ds,validation_data=val_ds,epochs=30,callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

Epoch 1/30
200/200 [==============================] - 4s 17ms/step - loss: 1.3804 - accuracy: 0.3347 - val_loss: 1.3704 - val_accuracy: 0.5281
Epoch 2/30
200/200 [==============================] - 3s 15ms/step - loss: 1.3546 - accuracy: 0.4631 - val_loss: 1.3347 - val_accuracy: 0.5506
Epoch 3/30
200/200 [==============================] - 3s 15ms/step - loss: 1.3060 - accuracy: 0.5506 - val_loss: 1.2768 - val_accuracy: 0.5831
Epoch 4/30
200/200 [==============================] - 3s 16ms/step - loss: 1.2390 - accuracy: 0.5895 - val_loss: 1.2061 - val_accuracy: 0.6275
Epoch 5/30
200/200 [==============================] - 3s 16ms/step - loss: 1.1639 - accuracy: 0.6367 - val_loss: 1.1304 - val_accuracy: 0.6619
Epoch 6/30
200/200 [==============================] - 3s 16ms/step - loss: 1.0851 - accuracy: 0.6777 - val_loss: 1.0595 - val_accuracy: 0.7038
Epoch 7/30
200/200 [==============================] - 4s 17ms/step - loss: 1.0132 - accuracy: 0.7094 - val_loss: 0.9953 - val_accuracy: 0.7300

In [ ]:
model.evaluate(test_ds)

250/250 [==============================] - 3s 13ms/step - loss: 0.5686 - accuracy: 0.7946


[0.5685995817184448, 0.7946249842643738]